# 📝 HOMEWORK 1 — Great Expectations (Amazon Sales Dataset)

Bu notebook, Amazon Sales dataseti üzerinde **Great Expectations Core v1.0+** kullanarak data quality validation yapar ve Slack'e bildirim gönderir.

## 🎯 Öğrenme Hedefleri
- GX Core v1.0+ code-first workflow kullanımı
- Ephemeral Data Context oluşturma (dosya yazmadan)
- Expectation Suite tanımlama
- Validation çalıştırma ve sonuçları yorumlama
- Slack entegrasyonu

## 📦 Dataset'teki Kasıtlı Hatalar

| Satır | Hata | Yakalanacak Expectation |
|-------|------|-------------------------|
| 8 | `Amount` boş (NULL) | ExpectColumnValuesToBeBetween |
| 9 | `ship-city` = "Chennai" (küçük harf) | ❌ Bonus challenge |
| 11 | `Order ID` boş (NULL) | ExpectColumnValuesToNotBeNull |
| 12 | `Qty` = -1 (negatif) | ExpectColumnValuesToBeBetween |
| 13 | `currency` = "USD", `ship-country` = "US" | ExpectColumnValuesToBeInSet |
| 14 | `Date` = "invalid-date" | ExpectColumnValuesToMatchRegex |

---
## 1️⃣ Kurulum

Gerekli kütüphaneleri yükleyelim.

In [ ]:
# Kurulum (sadece ilk seferde çalıştırın)
#%pip install great_expectations pandas requests
%pip install -U numpy

In [1]:
# Import'lar
import great_expectations as gx
import pandas as pd
import requests
import json
from datetime import datetime

# GX versiyonunu kontrol et (v1.0+ olmalı)
print(f"Great Expectations Version: {gx.__version__}")

Great Expectations Version: 1.11.3


---
## 2️⃣ Konfigürasyon

Validation kuralları için beklenen değerleri tanımlayalım.

In [ ]:
# Dosya yolu
CSV_PATH = "amazon_sales.csv"

# Slack Webhook URL (kendi URL'inizi environment variable olarak ayarlayın)
import os
SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL", "YOUR_SLACK_WEBHOOK_URL")

# Beklenen değerler
VALID_STATUSES = [
    "Cancelled",
    "Shipped",
    "Shipped - Delivered to Buyer",
    "Pending",
    "Shipped - Returned to Seller",
    "Shipped - Rejected by Buyer",
    "Shipped - Returning to Seller",
    "Shipped - Out for Delivery",
    "Shipped - Picked Up"
]

VALID_FULFILMENT = ["Merchant", "Amazon"]
VALID_CURRENCIES = ["INR"]
VALID_COUNTRIES = ["IN"]

---
## 3️⃣ Veriyi Yükle ve İncele

Önce datasetin yapısını anlayalım.

In [3]:
# CSV'yi yükle
df = pd.read_csv(CSV_PATH)

print(f"📊 Dataset Boyutu: {df.shape[0]} satır, {df.shape[1]} sütun")
print(f"\n📋 Sütunlar: {list(df.columns)}")

📊 Dataset Boyutu: 128975 satır, 24 sütun

📋 Sütunlar: ['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ', 'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN', 'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city', 'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids', 'B2B', 'fulfilled-by', 'Unnamed: 22']


C:\Users\uzumm\AppData\Local\Temp\ipykernel_18552\1551904776.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(CSV_PATH)


In [4]:
# İlk birkaç satıra bakalım
df.head(10)

,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
5,5,404-1490984-4578765,04-30-22,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,...,INR,824.00,GHAZIABAD,UTTAR PRADESH,201102.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
6,6,408-5748499-6859555,04-30-22,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,...,INR,653.00,CHANDIGARH,CHANDIGARH,160036.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
7,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3405,JNE3405-KR-S,kurta,...,INR,399.00,HYDERABAD,TELANGANA,500032.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
8,8,407-5443024-5233168,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,...,NaN,NaN,HYDERABAD,TELANGANA,500008.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
9,9,402-4393761-0311520,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3461,JNE3461-KR-XXL,kurta,...,INR,363.00,Chennai,TAMIL NADU,600041.0,IN,NaN,False,NaN,NaN


In [5]:
# Veri tipleri ve null değerler
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   index               128975 non-null  int64  
 1   Order ID            128975 non-null  object 
 2   Date                128975 non-null  object 
 3   Status              128975 non-null  object 
 4   Fulfilment          128975 non-null  object 
 5   Sales Channel       128975 non-null  object 
 6   ship-service-level  128975 non-null  object 
 7   Style               128975 non-null  object 
 8   SKU                 128975 non-null  object 
 9   Category            128975 non-null  object 
 10  Size                128975 non-null  object 
 11  ASIN                128975 non-null  object 
 12  Courier Status      122103 non-null  object 
 13  Qty                 128975 non-null  int64  
 14  currency            121180 non-null  object 
 15  Amount              121180 non-nul

In [6]:
# Temel istatistikler
df.describe()

,index,Qty,Amount,ship-postal-code
count,128975.000000,128975.000000,121180.000000,128942.000000
mean,64487.000000,0.904431,648.561465,463966.236509
std,37232.019822,0.313354,281.211687,191476.764941
min,0.000000,0.000000,0.000000,110001.000000
25%,32243.500000,1.000000,449.000000,382421.000000
50%,64487.000000,1.000000,605.000000,500033.000000
75%,96730.500000,1.000000,788.000000,600024.000000
max,128974.000000,15.000000,5584.000000,989898.000000


### 🔍 Soru: Yukarıdaki çıktılarda herhangi bir sorun görüyor musunuz?

*Düşünün ve aşağıda validation ile kontrol edelim...*

---
## 4️⃣ Great Expectations Context Oluştur

GX Core v1.0+ ile **Ephemeral Data Context** kullanıyoruz. Bu, herhangi bir dosya yazmadan bellekte çalışır.

In [7]:
# Ephemeral Data Context oluştur (dosya yazmaz!)
context = gx.get_context()

# Context tipini doğrula
print(f"✅ Context Type: {type(context).__name__}")

✅ Context Type: EphemeralDataContext


---
## 5️⃣ Data Source ve Data Asset Tanımla

GX'e verimizi tanıtalım.

In [8]:
# Pandas Data Source ekle
data_source = context.data_sources.add_pandas(name="amazon_sales_source")
print(f"✅ Data Source oluşturuldu: {data_source.name}")

# DataFrame Asset ekle
data_asset = data_source.add_dataframe_asset(name="amazon_sales_asset")
print(f"✅ Data Asset oluşturuldu: {data_asset.name}")

# Batch Definition oluştur (tüm DataFrame'i tek batch olarak al)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="full_dataframe")
print(f"✅ Batch Definition oluşturuldu: {batch_definition.name}")

✅ Data Source oluşturuldu: amazon_sales_source
✅ Data Asset oluşturuldu: amazon_sales_asset
✅ Batch Definition oluşturuldu: full_dataframe


---
## 6️⃣ Expectation Suite Oluştur

Şimdi data quality kurallarımızı (Expectations) tanımlayalım.

In [9]:
# Expectation Suite oluştur
suite = gx.ExpectationSuite(name="amazon_sales_suite")

# Suite'i context'e ekle
suite = context.suites.add(suite)
print(f"✅ Expectation Suite oluşturuldu: {suite.name}")

✅ Expectation Suite oluşturuldu: amazon_sales_suite


### 6.1 Order ID Kontrolleri

In [10]:
# Order ID: NULL olmamalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="Order ID")
)
print("✅ Expectation eklendi: Order ID - Not Null")

# Order ID: Unique olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeUnique(column="Order ID")
)
print("✅ Expectation eklendi: Order ID - Unique")

✅ Expectation eklendi: Order ID - Not Null
✅ Expectation eklendi: Order ID - Unique


### 6.2 Qty (Miktar) Kontrolü

In [11]:
# Qty: 0 veya daha büyük olmalı (negatif olamaz)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Qty",
        min_value=0,
        max_value=None  # Üst limit yok
    )
)
print("✅ Expectation eklendi: Qty >= 0")

✅ Expectation eklendi: Qty >= 0


### 6.3 Amount (Tutar) Kontrolü

In [12]:
# Amount: 0 veya daha büyük olmalı
# Not: NULL değerler de başarısız sayılacak
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Amount",
        min_value=0,
        max_value=None
    )
)
print("✅ Expectation eklendi: Amount >= 0")

✅ Expectation eklendi: Amount >= 0


### 6.4 Kategorik Değer Kontrolleri

In [13]:
# Status: Belirlenen değerler içinde olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="Status",
        value_set=VALID_STATUSES
    )
)
print("✅ Expectation eklendi: Status - Valid Values")

# Fulfilment: Merchant veya Amazon olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="Fulfilment",
        value_set=VALID_FULFILMENT
    )
)
print("✅ Expectation eklendi: Fulfilment - Valid Values")

# Currency: INR olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="currency",
        value_set=VALID_CURRENCIES
    )
)
print("✅ Expectation eklendi: Currency = INR")

# Ship Country: IN olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="ship-country",
        value_set=VALID_COUNTRIES
    )
)
print("✅ Expectation eklendi: Ship Country = IN")

✅ Expectation eklendi: Status - Valid Values
✅ Expectation eklendi: Fulfilment - Valid Values
✅ Expectation eklendi: Currency = INR
✅ Expectation eklendi: Ship Country = IN


### 6.5 Tarih Format Kontrolü

In [15]:
df['Date'].head(1)

0    04-30-22
Name: Date, dtype: object

In [16]:
# Date: MM-DD-YY formatında olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToMatchRegex(
        column="Date",
        regex=r"^\d{2}-\d{2}-\d{2}$"  # MM-DD-YY
    )
)
print("✅ Expectation eklendi: Date Format (MM-DD-YY)")

✅ Expectation eklendi: Date Format (MM-DD-YY)
The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [17]:
# Toplam expectation sayısı
print(f"\n📋 Toplam Expectation Sayısı: {len(suite.expectations)}")


📋 Toplam Expectation Sayısı: 9


---
## 7️⃣ Validation Definition Oluştur ve Çalıştır

In [18]:
# Validation Definition oluştur
validation_definition = gx.ValidationDefinition(
    name="amazon_sales_validation",
    data=batch_definition,
    suite=suite
)

# Context'e ekle
validation_definition = context.validation_definitions.add(validation_definition)
print(f"✅ Validation Definition oluşturuldu: {validation_definition.name}")

✅ Validation Definition oluşturuldu: amazon_sales_validation


In [19]:
# Validation'ı çalıştır
print("🔄 Validation çalıştırılıyor...\n")

results = validation_definition.run(
    batch_parameters={"dataframe": df}
)

print("✅ Validation tamamlandı!")

🔄 Validation çalıştırılıyor...



Calculating Metrics: 100%|██████████| 65/65 [00:01<00:00, 37.63it/s] 

✅ Validation tamamlandı!


---
## 8️⃣ Sonuçları İncele

In [ ]:
# Genel sonuç
print("=" * 60)
print("   VALIDATION SONUCU")
print("=" * 60)

if results.success:
    print("\n   🎉 TÜM KONTROLLER BAŞARILI!")
else:
    print("\n   ❌ BAZI KONTROLLER BAŞARISIZ!")
    
print("=" * 60)



   VALIDATION SONUCU

   ❌ BAZI KONTROLLER BAŞARISIZ!


In [21]:
# Detaylı sonuçları işle
results_dict = results.to_json_dict()
expectation_results = results_dict.get("results", [])

passed = []
failed = []

for exp_result in expectation_results:
    exp_config = exp_result.get("expectation_config", {})
    exp_type = exp_config.get("type", "Unknown")
    column = exp_config.get("kwargs", {}).get("column", "N/A")
    success = exp_result.get("success", False)
    result_detail = exp_result.get("result", {})
    
    info = {
        "expectation": exp_type,
        "column": column,
        "success": success,
        "result": result_detail
    }
    
    if success:
        passed.append(info)
    else:
        failed.append(info)

print(f"\n📊 Özet:")
print(f"   ✅ Başarılı: {len(passed)}")
print(f"   ❌ Başarısız: {len(failed)}")
print(f"   📋 Toplam: {len(expectation_results)}")


📊 Özet:
   ✅ Başarılı: 7
   ❌ Başarısız: 2
   📋 Toplam: 9


In [22]:
# Başarısız kontrolleri detaylı göster
if failed:
    print("\n" + "=" * 60)
    print("   ❌ BAŞARISIZ KONTROLLER")
    print("=" * 60)
    
    for f in failed:
        print(f"\n📌 {f['expectation']}")
        print(f"   Sütun: {f['column']}")
        
        result = f.get('result', {})
        unexpected_count = result.get('unexpected_count', 0)
        unexpected_percent = result.get('unexpected_percent', 0)
        unexpected_values = result.get('partial_unexpected_list', [])
        
        if unexpected_count:
            print(f"   Hatalı Kayıt Sayısı: {unexpected_count}")
            print(f"   Hatalı Kayıt Oranı: {unexpected_percent:.2f}%")
        if unexpected_values:
            print(f"   Örnek Hatalı Değerler: {unexpected_values[:5]}")
else:
    print("\n🎉 Tüm kontroller başarılı!")


   ❌ BAŞARISIZ KONTROLLER

📌 expect_column_values_to_be_unique
   Sütun: Order ID
   Hatalı Kayıt Sayısı: 15443
   Hatalı Kayıt Oranı: 11.97%
   Örnek Hatalı Değerler: ['403-4367956-2849158', '403-4367956-2849158', '404-2262140-4696366', '404-2262140-4696366', '408-4069830-3819562']

📌 expect_column_values_to_be_in_set
   Sütun: Status
   Hatalı Kayıt Sayısı: 295
   Hatalı Kayıt Oranı: 0.23%
   Örnek Hatalı Değerler: ['Shipped - Lost in Transit', 'Shipped - Lost in Transit', 'Shipped - Lost in Transit', 'Shipped - Lost in Transit', 'Shipped - Lost in Transit']


In [23]:
# Başarılı kontrolleri göster
if passed:
    print("\n" + "=" * 60)
    print("   ✅ BAŞARILI KONTROLLER")
    print("=" * 60)
    
    for p in passed:
        print(f"   ✓ {p['expectation']} (Sütun: {p['column']})")


   ✅ BAŞARILI KONTROLLER
   ✓ expect_column_values_to_not_be_null (Sütun: Order ID)
   ✓ expect_column_values_to_be_between (Sütun: Qty)
   ✓ expect_column_values_to_be_between (Sütun: Amount)
   ✓ expect_column_values_to_be_in_set (Sütun: Fulfilment)
   ✓ expect_column_values_to_be_in_set (Sütun: currency)
   ✓ expect_column_values_to_be_in_set (Sütun: ship-country)
   ✓ expect_column_values_to_match_regex (Sütun: Date)


---
## 9️⃣ Slack Bildirimi Gönder

Validation sonuçlarını Slack'e gönderelim.

In [24]:
def send_slack_notification(webhook_url, overall_success, passed_count, failed_count, failed_list):
    """
    Slack'e validation sonuçlarını gönder.
    """
    if webhook_url == "YOUR_SLACK_WEBHOOK_URL":
        print("⚠️  Slack bildirimi atlandı (webhook URL yapılandırılmamış)")
        print("   Kendi Slack Webhook URL'inizi SLACK_WEBHOOK_URL değişkenine yazın.")
        return False
    
    # Emoji ve renk
    if overall_success:
        emoji = "✅"
        color = "#36a64f"
        status_text = "PASSED"
    else:
        emoji = "❌"
        color = "#dc3545"
        status_text = "FAILED"
    
    # Failed detayları
    failed_details = ""
    if failed_list:
        failed_details = "\n".join([
            f"• *{f['expectation']}* (Sütun: `{f['column']}`)"
            for f in failed_list
        ])
    
    # Mesaj oluştur
    message = {
        "attachments": [
            {
                "color": color,
                "blocks": [
                    {
                        "type": "header",
                        "text": {
                            "type": "plain_text",
                            "text": f"{emoji} Data Quality Validation {status_text}",
                            "emoji": True
                        }
                    },
                    {
                        "type": "section",
                        "fields": [
                            {"type": "mrkdwn", "text": f"*Dataset:*\nAmazon Sales"},
                            {"type": "mrkdwn", "text": f"*Zaman:*\n{datetime.now().strftime('%Y-%m-%d %H:%M')}"},
                            {"type": "mrkdwn", "text": f"*Başarılı:*\n{passed_count} ✓"},
                            {"type": "mrkdwn", "text": f"*Başarısız:*\n{failed_count} ✗"}
                        ]
                    }
                ]
            }
        ]
    }
    
    # Failed varsa ekle
    if failed_details:
        message["attachments"][0]["blocks"].append({
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"*Başarısız Kontroller:*\n{failed_details}"
            }
        })
    
    # Gönder
    try:
        response = requests.post(
            webhook_url,
            data=json.dumps(message),
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code == 200:
            print("✅ Slack bildirimi başarıyla gönderildi!")
            return True
        else:
            print(f"❌ Slack bildirimi başarısız: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ Slack bildirimi hatası: {e}")
        return False

In [25]:
# Slack bildirimi gönder
send_slack_notification(
    webhook_url=SLACK_WEBHOOK_URL,
    overall_success=results.success,
    passed_count=len(passed),
    failed_count=len(failed),
    failed_list=failed
)

✅ Slack bildirimi başarıyla gönderildi!


True

---
## 🔟 CI/CD için Exit Code

GitHub Actions veya Jenkins'te kullanmak için:

In [ ]:
# CI/CD için (bu hücreyi script olarak çalıştırırken kullanın)
# if not results.success:
#     print("⚠️ Validation başarısız! Exit code: 1")
#     # exit(1)  # Uncomment for CI/CD
# else:
#     print("🎉 Tüm validasyonlar başarılı!")
#     # exit(0)  # Uncomment for CI/CD

---
## 📚 Özet

Bu notebook'ta öğrendiklerimiz:

1. **GX Core v1.0+ API** kullanımı (code-first, dosyasız)
2. **Ephemeral Data Context** oluşturma
3. **Data Source → Data Asset → Batch Definition** zinciri
4. **Expectation Suite** tanımlama
5. **Validation Definition** ile validation çalıştırma
6. **Slack entegrasyonu** ile bildirim gönderme

### 🏋️ Bonus Alıştırmalar

1. `ship-city` sütunundaki büyük/küçük harf tutarsızlığını yakalayacak bir expectation ekleyin
2. `ship-postal-code` için 6 haneli format kontrolü ekleyin
3. `Status` = "Cancelled" iken `Qty` = 0 olmalı kuralını ekleyin (compound expectation)

---
## 🔗 Kaynaklar

- [GX Core Documentation](https://docs.greatexpectations.io/docs/core/introduction/)
- [Try GX Core](https://docs.greatexpectations.io/docs/core/introduction/try_gx/)
- [Connect to DataFrame Data](https://docs.greatexpectations.io/docs/core/connect_to_data/dataframes/)
- [Expectation Gallery](https://greatexpectations.io/expectations/)